# 10. 고유 아이디, 사이드 이펙트

<br>

<hr>

<br>

## 01. `useId` 훅
- React에서 컴포넌트를 생성할 때, UI요소에 고유한 ID를 부여하는 일은 중요
  
  $\rightarrow$ **ID 값을 수동으로 관리하는 일은 번거로우며 중복 등의 오류 발생 가능성**

- React 16.8부터 훅이 도입되었지만, 당시 고유한 ID를 자동으로 생성하는 기능이 내장 X
  
  $\rightarrow$ [`uuid`](https://github.com/uuidjs/uuid)와 같은 서드 파티 라이브러리를 활용

  $\rightarrow$ **React 18부터 `useId`훅이 새롭게 추가되며, 별도 컴포넌트마다 고유한 ID 값을 생성하고 관리 가능**

<br>

#### `useId` 훅
- 컴포넌트 내부에서 자동으로 고유한 ID 생성

```ts
const id = useId();
```

<br>

- 생성한 ID를 `<input>` 요소의 `id` 속성과 `<label>` 요소의 `htmlFor` 속성에 함께 사용하면 컴포넌트를 여러개 렌더링해도 ID 충돌 문제가 발생하지 않음
- `components/Input.tsx`

```ts
import { useId } from "react";

type InputProps = React.ComponentPropsWithRef<'input'>;

export default function Input({ children, ...props} : InputProps) {
    const uuid = useId();
    return (
        <>
            <label htmlFor={uuid}>{children}</label>
            <input type="text" id={uuid} {...props} />
        </>
    )
}
```

<br>

- `App.tsx`

```ts
import Input from "./components/Input";

export default function App() {
  return (
    <>
      <Input type="email" placeholder="Enter Your Email">이메일</Input>
      <Input type="password" placeholder="Enter Your Password">비밀번호</Input>
    </>
  );
}
```

<img src='img/0905.png' width=400>

<br>

<hr>

<br>

## 02. `useEffect`훅
- React에서 컴포넌트의 기본 역할은 화면을 렌더링하는 것 이외에,
  
  **외부 API 호출, 로컬 스토리지 $\cdot$쿠키에 데이터를 저장하거나, 외부 라이브러리와 상호작용 하는 작업을 수행** = **사이드 이펙트**

- React에서는 사이드 이펙트를 처리하기 위해 `useEffect` 훅을 사용
  - 컴포넌트가 화면에 렌더링된 이후에 (UI가 실제로 DOM에 그려진 뒤) 실행되는 부수적인 작업을 처리할 수 있도록 도와줌
  
<br>

- **`useEffect` 훅은 다음과 같이 세 가지 시점에서 사이드 이펙트를 처리**
  - **마운트 (mount, 생성)** : 컴포넌트가 처음 생성되어 화면에 나타날 때
  - **업데이트 (update, 수정)** : 상태나 `props`가 변경되어 컴포넌트가 리렌더링
  - **언마운트 (unmount, 소멸)** : 컴포넌트가 화면에서 사라질 떄

- **`useEffect` 훅은 컴포넌트의 생명주기 각 단계에서 필요한 작업을 수행할 수 있게 함**
  - **컴포넌트의 생명주기도 사이드 이펙트의 한 종류**
  - **`useEffect` 훅은 사이드 이펙트 처리 뿐만 아니라 컴포넌트가 언마운트될 때 정리(clean-up) 작업도 수행할 수 있음**

<br>

- **`useEffect` 훅의 기본 형식**
    - **`setup` (설정 함수) : `useEffect` 훅이 실행할 로직을 담은 함수**
      - React는 이 함수를 컴포넌트가 DOM에 렌더링된 후에 실행
      - 선택적으로 Clean-up 함수를 반환할 수  있으며, 의존성 배열의 값이 변경되면, 이전 값을 기반으로 Clean-up 함수를 먼저 실행한 다음, 새로운 값으로 설정한 함수를 다시 실행
      - 컴포넌트가 언마운트 될 때도 Clean-up 함수가 호출
    - **`dependencies` (의존성 배열) : 설정 함수 안에서 참조하는 모든 반응형 값 (상태, `props`, 함수 등)을 나열한 배열**
      - React는 배열에 있는 값들이 이전 렌더링과 비교해 변경되었는지를 판단하고, 변경되었을 때만 설정 함수를 다시 실행

        이때 내부적으로 `Object.is` 알고리즘을 사용해 값을 비교

      - 의존성 배열을 생략하면 컴포넌트가 렌더링될 때마다 `useEffect` 훅이 실행
      - `ESLint` 플러그인을 사용하면 의존성 배열에 필요한 값이 빠지지 않았는지 자동으로 검사

```ts
useEffect(setup, dependencies)
```

<br>

#### `Object.is`
- **`Object.is`는 숫자의 부호 차이나 `NaN`과 같은 예외적인 값까지 구분할 수 있기 때문에 `useEffect`훅의 `dependencies`배열을 비교할 때 더 정확하게 판단 가능**
- 예
  - `Object.is(NaN, NaN)`은 `true`이지만, `NaN === NaN`은 `false`
  - `Object.is(+0, -0)`은 `false`이지만, `+0 === -0`은 `true`

<br>

### 컴포넌트의 생명주기
- **React의 모든 컴포넌트는 '마운트(생성) $\rightarrow$ 업데이트(수정) $\rightarrow$ 언마운트(소멸) 과정'이라는 생명주기 (LifeCycle)를 가짐** 

<br>

#### 마운트
- 컴포넌트가 마운트되는 시점에 특정 사이드 이펙트를 실행
- **`useEffect` 훅의 첫 번째 인자인 설정 함수는 항상 콜백 함수 형태로 작성해야 함**
- **설정 함수가 마운트 시점에만 실행되도록 하려면, 두 번째 인자인 의존성 배열을 빈 배열 (`[]`)로 설정해야 함**
  
  $\rightarrow$ 컴포넌트가 처음 화면에 렌더링될 때 단 한번만 콜백 함수가 실행

<br>

```ts
import { useEffect } from 'react';

export default function Mount() {
    useEffect(() => {
        console.log('Mounted');
    }, []);
    return <div>Mount</div>
}
```

<br>

#### 언마운트
- 컴포넌트가 언마운트되는 시점에 특정 사이드 이펙트를 실행
  - **`useEffect` 훅 안의 설정 함수 안에서 또 다른 함수를 반환할 수 있으며, 반환한 함수는 컴포넌트가 화면에서 사라질 때 (언마운트 시점) 자동으로 실행 됨**  
- 언마운트 시점을 활용할 때도 의존성 배열을 빈 배열 (`[]`)로 설정
  
  **의존성 배열에 상태 값을 포함하면 해당 값이 바뀔 때 마다 `useEffect` 훅의 설정 함수가 다시 실행되기 때문**

  $\rightarrow$ **업데이트 시점에도 사이드 이펙트가 발생**

<br>

```ts
import { useEffect } from 'react';

export default function UnMount() {
    
    useEffect(() => {
        return () => {
            console.log('UnMounted');
        };
    }, []);

    return <div>UnMount</div>
}
```

<br>

#### 업데이트
- 컴포넌트가 업데이트 되는 시점에 특정 사이드 이펙트를 실행
  - **업데이트 : 컴포넌트의 상태가 변경되는 것**
- **업데이트 시점의 `useEffect` 훅이 마운트 시점에도 실행 됨**
  
  $\rightarrow$ **의존성 배열에 값이 포함되어 있든 없든 `useEffect` 훅은 컴포넌트가 처음 렌더링되는 시점에 설정 함수는 반드시 한 번 실행됨**

  $\rightarrow$ **초기화 작업 혹은 처음 한 번만 실행되어야 하는 로직은 보통 의존성 배열을 빈 배열로 설정해 마운트 시점에만 실행되도록 작성**

<br>

```ts
import { useEffect, useState } from "react";

export default function Update() {
    const [count, setCount] = useState(0);
    useEffect(() => {
        console.log('Updated : ', `${count}`);
    }, [count]);

    return (
        <>
            <h1>Count: {count}</h1>
            <button onClick={() => setCount((count) => count + 1)}>증가</button>
        </>
    )
}
```

<br>



### `useEffect` 훅 사례

<br>

#### API 호출
- `useEffect` 훅의 가장 대표적인 사례
  
  **외부 API를 통해 데이터를 가져오는 작업 (`fetch`)**
- React는 기본적으로 클라이언트 사이드 렌더링 (CSR) 방식을 사용
  
  $\rightarrow$ 서버에서 데이터를 가져오는 작업은 웹 브라우저 (클라이언트)에서 직접 수행

  $\rightarrow$ **이러한 데이터 요청은 화면 렌더링과 직접적인 관계는 없지만, 컴포넌트가 동작하는 데 필요하므로, React에서는 사이드 이펙트로 분류하고 `useEffect` 훅을 통해 처리**

<br>

```ts
import { useEffect } from "react";

export default function FetchUser() {
    useEffect(() => {
        fetch('https://jsonplaceholder.typicode.com/users') //  외부 API에 GET 요청
            .then((response) => response.json()) // 응답을 JSON으로 파싱
            .then((data) => console.log(data));
    }, []);

    return <div>FetchUser</div>
}
```

<br>

#### 타이머 설정
- **컴포넌트에서 `setTimeout()`이나 `setInterval()` 함수를 사용해 시간을 기반으로 작동하는 작업을 설정할 때도 `useEffect`훅이 중요한 역할**

<br>

- **컴포넌트가 마운트된 이후 매초 숫자를 증가시키는 타이머**
    - 타이머는 컴포넌트가 화면에서 사라질 때 반드시 정리 $\rightarrow$ `clearInterval()`함수로 타이머 제거

```ts
import { useState, useEffect } from "react";

export default function Timer() {
    const [seconds, setSeconds] = useState(0);

    useEffect(() => {
        const interval = setInterval(() => {
            setSeconds((prev) => prev + 1);
        }, 1000); // 1초마다 seconds 값을 1씩 증가
        return () => clearInterval(interval); // 언마운트시 타이머 정리
    }, []); // 마운트 시점에 한번만 실행

    return <p>timer: {seconds} seconds</p>
}
```

<br>

#### 실시간 이벤트 처리
- 일부 이벤트는 JSX 요소의 `onClick`, `onChange`와 같은 이벤트 속성으로 간단히 처리 가능하지만,
  
  **컴포넌트 전체를 대상으로 이벤트를 감지해야 하는 경우, `useEffect` 훅을 사용해 이벤트 리스너를 등록하는 방식이 효과적**


<br>

- **스크롤 위치 추적**

```ts
import { useEffect } from "react";

export default function ScrollTracker() {
    useEffect(() => {
        const handleScroll = () => {
            console.log('현재 스크롤 위치', window.scrollY);
        };
        window.addEventListener('scroll', handleScroll); // 스크롤 이벤트를 등록
        
        return () => {
            window.removeEventListener('scroll', handleScroll)
        }; // 언마운트 시 이벤트 제거
    }, []); // 마운트시 한 번만 실행

    return <div style={{height: '200vh'}}>스크롤</div>
}
```

<img src='img/0906.png' width=400>

<br>

#### 자동 저장 기능

<br>


- **일정 시간마다 입력 값을 로컬 저장소에 자동 저장해두면 사용자가 페이지를 새로 고침해도 이전에 입력한 내용을 복구**
  - **첫 번째 `useEffect`** : 마운트될 때 실행
    - 로컬 저장소 (`localStorage`)에 저장된 값이 있다면 불러와서 `formData`상태에 설정
  - **두 번째 `useEffect`** : `formData` 상태가 변경될 때마다 실행
    - 입력 값이 변경되면 1초 뒤에 `localStorage`에 자동으로 저장되도록 `setTimeout()`을 사용
    - 저장 직전에 이전 타이머를 `clearTimeout()`으로 제거해 불필요한 중복 저장을 방지

<br>

```ts
import { useEffect, useState } from "react";

export default function AutoSaveForm() {
    const [formData, setFormData] = useState('');


    useEffect(() => {
        const savedData = localStorage.getItem('savedFormData');

        if (savedData) {
            setFormData(savedData);
        }
    }, []);

    useEffect(() => {
        const timeoutId = setTimeout(() => {
            localStorage.setItem('savedFormData', formData);
        }, 1000);

        return () => clearTimeout(timeoutId);
    }, [formData]);

    return (
        <textarea
        value={formData}
        onChange={(e) => setFormData(e.target.value)}
        placeholder="Autosave"
        ></textarea>
    )
}
```

<br>

#### 실시간 통신 기능 구현
- 실시간 기능이 필요한 웹 애플리케이션에서는 웹소켓(Web Socket)을 활용해 서버와의 양방향 통신을 구현하는 경우가 존재
  - 예) 채팅, 온라인 회의 등 
  
  $\rightarrow$ `useEffect` 혹은 웹 소켓 연결을 생성하고 정리하는 데 중요한 역할

<br>

```ts
import { useEffect, useState } from "react";

export default function WebSocketTest() {
  const [messages, setMessages] = useState<string[]>([]);
  const [message, setMessage] = useState<string>("");
  const [socket, setSocket] = useState<WebSocket | null>(null);

  useEffect(() => {
    // 마운트 시 웹소켓 연결 생성
    const socket = new WebSocket("wss://echo.websocket.org");
    setSocket(socket);

    socket.onmessage = (event) => {
      // 서버로부터 메시지를 받으면 화면에 출력
      setMessages((prev) => [...prev, `서버: ${event.data}`]);
    };

    socket.onerror = (error) => {
      console.error("웹소켓 오류:", error);
    };

    socket.onclose = () => {
      console.log("웹소켓 연결 종료");
    };

    return () => {
      socket.close(); // 언마운트 시 웹소켓 연결 정리
    };

  }, []);

  const handleSendMessage = () => {
    // 사용자 메시지를 서버로 전송하고 화면에 표시
    if (socket && socket.readyState === WebSocket.OPEN && message) {
      socket.send(message);
      setMessages((prev) => [...prev, `나: ${message}`]);
      setMessage("");
    } else {
      alert("서버 연결이 끊겼습니다.");
    }
  };

  return (
    <div>
      <div>
        {messages.map((msg, index) => (
          <div key={index} className="message">
            {msg}
          </div>
        ))}
      </div>
      <div>
        <input
          type="text"
          value={message}
          onChange={(e) => setMessage(e.target.value)}
          placeholder="메시지를 입력하세요."
        />
        <button onClick={handleSendMessage}>전송</button>
      </div>
    </div>
  );
}
```

<br>

<hr>

<br>


## 03. 할 일 관리 애플리케이션 개선

<br>

### 폼 요소 연결하기
- **웹 접근성과 사용자 편의성을 높이기 위해서 `<input>` 태그와 `<label>`태그를 각각 `id` 속성과 `htmlFor` 속성으로 연결하는 방식으로 개선하는 것이 바람직**
  
  $\rightarrow$ **`useId` 훅을 사용해 체크박스를 개선**

<br>

- `components/html/Checkbox.tsx`
    - `useId()` 훅을 사용해 컴포넌트마다 고유한 ID를 자동으로 생성

```ts
import { useId } from "react";

type CheckboxProps = Omit<React.ComponentPropsWithRef<'input'>, 'type'> & {
    type?: 'checkbox';
    parentClassName: string;
}

export default function Checkbox(props: CheckboxProps) {
    const { parentClassName, children, ...rest } = props;
    const uuid = useId();

    return (
        <div className={parentClassName}>
            <input id={uuid}  { ...rest } />
            <label htmlFor={uuid}>{children}</label>
        </div>
    )
}
```

<br>

### 할 일 저장하기
- `Storage API`를 사용해 등록한 할 일을 저장
  - `Storage API` : 웹 브라우저 안에 데이터를 저장하고 꺼내 쓸 수 있도록 도와주는 기능
    - `sessionStorage` : 웹 브라우저의 탭을 닫을 떄까지 데이터 유지
    - `localStorage` : 웹 브라우저를 닫았다가 열어도 데이터 유지
- **`Storage API`를 사용하는 작업은 렌더링 외 동작, 즉 사이드 이펙트**

<br>

- **`App.tsx`**
  - **애플리케이션을 처음 실행할 때 `localStorage`에 저장된 할 일 목록을 불러와 `todos`의 초깃값으로 사용** 
    - **저장된 데이터가 없으면 빈 배열 (`[]`)을 기본값으로 사용**
  - **`todos` 상태가 변경될 때마다 `localStorage`에 다시 저장**

```ts
export default function App() {

  const [todos, setTodos] = useState<Todo[]>(() => JSON.parse(localStorage.getItem('todos') || '[]')); 

  // 저장된 totods 목록 불러오기

  ...

  // todos 상태 값이 변경될 때마다 localStorage에 저장
  useEffect(() => {
    localStorage.setItem('todos', JSON.stringify(todos));
  }, [todos]);

  return (

    ...

  )
}
```

<br>

<hr>